## Startup

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hmmlearn import hmm

import os
import pickle

In [29]:
hmm.logging.disable(level=80)

In [27]:
print(dir(hmm.logging))

['BASIC_FORMAT', 'BufferingFormatter', 'CRITICAL', 'DEBUG', 'ERROR', 'FATAL', 'FileHandler', 'Filter', 'Filterer', 'Formatter', 'GenericAlias', 'Handler', 'INFO', 'LogRecord', 'Logger', 'LoggerAdapter', 'Manager', 'NOTSET', 'NullHandler', 'PercentStyle', 'PlaceHolder', 'RootLogger', 'StrFormatStyle', 'StreamHandler', 'StringTemplateStyle', 'Template', 'WARN', 'WARNING', '_STYLES', '_StderrHandler', '__all__', '__author__', '__builtins__', '__cached__', '__date__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__status__', '__version__', '_acquireLock', '_addHandlerRef', '_checkLevel', '_defaultFormatter', '_defaultLastResort', '_handlerList', '_handlers', '_is_internal_frame', '_levelToName', '_lock', '_logRecordFactory', '_loggerClass', '_nameToLevel', '_register_at_fork_reinit_lock', '_releaseLock', '_removeHandlerRef', '_showwarning', '_srcfile', '_startTime', '_str_formatter', '_warnings_showwarning', 'addLevelName', 'atexit', 'basicConfig

In [2]:
random_state=42
np.random.seed(random_state)

In [3]:
from scripts.params import get_params

params = get_params()

## Data Retrieval

In [4]:
dataroute=os.path.join("..",  "data")
dumproute=os.path.join("..",  "dump")
resultsroute=os.path.join("..",  "results")

In [5]:
name=f'finaldf_train_{params["tablename"]}.pickle'
filename=os.path.join(dataroute, name)
with open(filename, 'rb') as handle:
    df=pickle.load(handle)


In [6]:
df.head()

,^MERV_rets,^MERV_log_rets,^MERV_gk_vol,GGAL.BA_rets,GGAL.BA_log_rets,GGAL.BA_gk_vol,GGAL_rets,GGAL_log_rets,GGAL_gk_vol,YPFD.BA_rets,...,BBAR.BA_gk_vol,BBAR_rets,BBAR_log_rets,BBAR_gk_vol,USD_rets,USD_log_rets,USD_gk_vol,USD_^MERV_rets,USD_^MERV_log_rets,USD_^MERV_gk_vol
2013-01-03,0.007552,0.007524,0.000129,0.010616,0.010560,0.000677,-0.012748,-0.012830,0.001228,-0.006862,...,0.000169,-0.005725,-0.005742,0.000960,0.008830,0.008792,0.000014,0.001247,0.001246,0.000129
2013-01-04,0.007092,0.007067,0.000158,-0.006303,-0.006323,0.000208,-0.010043,-0.010094,0.000554,0.004936,...,0.000406,-0.019194,-0.019381,0.000635,0.018043,0.017883,0.000133,-0.005727,-0.005744,0.000158
2013-01-07,-0.001035,-0.001035,0.000022,0.002114,0.002112,0.000063,-0.014493,-0.014599,0.000517,0.010805,...,0.000492,0.015656,0.015534,0.000511,-0.002489,-0.002492,0.000048,-0.009769,-0.009817,0.000022
2013-01-08,0.008285,0.008251,0.000082,-0.008439,-0.008475,0.000153,-0.016176,-0.016309,0.001085,0.049563,...,0.000438,-0.015414,-0.015534,0.000642,0.015356,0.015239,0.000064,-0.001117,-0.001118,0.000082
2013-01-09,0.017826,0.017669,0.000273,0.000000,0.000000,0.000000,0.011958,0.011887,0.005238,0.000000,...,0.000000,-0.003914,-0.003921,0.000147,-0.008671,-0.008709,0.001065,0.017245,0.017098,0.000273


In [7]:
tickerlist=params["tickerlist"]

## HMM Training

In [8]:
range_states=range(1,16)
emptydf=pd.DataFrame(columns=["AIC", "BIC"], index=range_states)
emptydf.fillna(np.inf, inplace=True)
results_dict_df={stock:emptydf for stock in tickerlist}

In [9]:
aic_best_model={stock:None for stock in tickerlist}
bic_best_model={stock:None for stock in tickerlist}

In [30]:
for stock in tickerlist:
    columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
    insample_data = df[columns]

    param_dict={
        "covariance_type" : "diag", 
        "n_iter" : 500,
        "random_state" : random_state
        #no voy a usar startprob_prior por devlog 20-06-23
        }

    for nstate in range_states:
        model = hmm.GaussianHMM(n_components= nstate, **param_dict, verbose=False)
        results = model.fit(insample_data)

        convergence=results.monitor_.converged
        # esta es la condición de si el modelo convergió

        all_states_found=np.isclose(a=(model.transmat_.sum(axis=1)), b=1).all()
        # esta es la condición de que todos los estados (nstates) hayan sido observados
        # si no, alguna fila en la matriz de transición del modelo son 0.
        # el errormsg es "Some rows of transmat_ have zero sum because no transition from the state was ever observed".

        startprob_check = (model.startprob_.sum()==1)
        # esta es la condición de que los estados al inicializar estén definidos
        
        good_model = convergence and all_states_found and startprob_check

        if good_model:
            try:
                results_dict_df[stock].loc[nstate, "AIC"]=model.aic(insample_data)
                results_dict_df[stock].loc[nstate, "BIC"]=model.bic(insample_data)
            except ValueError:
                pass
        else: 
            print(">"*10,f"{stock} {nstate} did not converge")
            results_dict_df[stock].loc[nstate, "BIC"]=np.inf
            results_dict_df[stock].loc[nstate, "BIC"]=np.inf

>>>>>>>>>> GGAL.BA 4 did not converge
>>>>>>>>>> GGAL.BA 5 did not converge


c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\hmmlearn\hmm.py:340: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])


>>>>>>>>>> GGAL 8 did not converge


c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\hmmlearn\hmm.py:340: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])


>>>>>>>>>> YPFD.BA 5 did not converge
>>>>>>>>>> YPFD.BA 8 did not converge
>>>>>>>>>> YPFD.BA 14 did not converge
>>>>>>>>>> YPFD.BA 15 did not converge
>>>>>>>>>> YPF 3 did not converge
>>>>>>>>>> YPF 5 did not converge
>>>>>>>>>> YPF 11 did not converge
>>>>>>>>>> EDN.BA 7 did not converge


...

>>>>>>>>>> BBAR.BA 5 did not converge
>>>>>>>>>> BBAR.BA 7 did not converge


c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\hmmlearn\hmm.py:340: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])


>>>>>>>>>> BBAR.BA 12 did not converge
>>>>>>>>>> BBAR.BA 13 did not converge
>>>>>>>>>> BBAR.BA 14 did not converge
>>>>>>>>>> BBAR 6 did not converge
>>>>>>>>>> BBAR 7 did not converge


c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\hmmlearn\hmm.py:340: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])


>>>>>>>>>> BBAR 10 did not converge


In [31]:
for stock in tickerlist:
    columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
    insample_data = df[columns]
    
    best_aic_nstate=results_dict_df[stock]["AIC"].astype(float).idxmin()
    best_bic_nstate=results_dict_df[stock]["BIC"].astype(float).idxmin()
    print(f"For stock {stock}, best AIC: {best_aic_nstate} best BIC: {best_bic_nstate}")

    aic_best_model[stock]=hmm.GaussianHMM(n_components = best_aic_nstate, **param_dict).fit(insample_data)
    bic_best_model[stock]=hmm.GaussianHMM(n_components = best_bic_nstate, **param_dict).fit(insample_data)

For stock ^MERV, best AIC: 6 best BIC: 3
For stock GGAL.BA, best AIC: 6 best BIC: 3
For stock GGAL, best AIC: 6 best BIC: 3
For stock YPFD.BA, best AIC: 6 best BIC: 3
For stock YPF, best AIC: 6 best BIC: 3
For stock EDN.BA, best AIC: 6 best BIC: 3
For stock EDN, best AIC: 6 best BIC: 3
For stock BMA.BA, best AIC: 6 best BIC: 3
For stock BMA, best AIC: 6 best BIC: 3
For stock BBAR.BA, best AIC: 6 best BIC: 3
For stock BBAR, best AIC: 6 best BIC: 3


# Generating out of sample data

In [32]:
name=f'finaldf_test_{params["tablename"]}.pickle'
filename=os.path.join(dataroute, name)
with open(filename, 'rb') as handle:
    df_test=pickle.load(handle)

LA SIGUIENTE FUNCION FUNCIONA  
EL PROBLEMA ES QUE CON ALGUNOS EXTERNOS FALLA  

https://github.com/alfsn/regime-switching-hmm/issues/42

In [36]:
def generate_HMM_samples_residuals(model, insample_data, oos_data):
    """_summary_

    Args:
        model (_type_): _description_
        insample_data (_type_): _description_
        oos_data (_type_): _description_
    """
    # pseudocodigo
    # agarra el mejor modelo (esto con una cantidad optima de params ya esta)
    # fittear t-j con t-j-252d
    # Darle un año de datos hasta t-j para que me prediga la secuencia (probabilidad) de estados.
        # Le pido que me prediga las probabilidades de cada estado durante el periodo t-j, t-j-252: 
        # esto me da una matriz de (252 x n estados)
        # esto entiendo es https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn.hmm.GaussianHMM.predict_proba
    # Tomo la ultima fila de la matriz
    # Multiplico esa por el vector de medias estimadas: este punto es mi forecast. 
        # esto es model.means_ (!)    
    nstate=model.n_components
    columns=oos_data.columns

    split_date = oos_data.index[0]
    dates_to_forecast = len(oos_data.index)

    probabilities=pd.DataFrame(columns=range(nstate), index=oos_data.index)
    forecasts=pd.DataFrame(columns=oos_data.columns, index=oos_data.index)

    oos_data = pd.concat([insample_data, oos_data])
    del insample_data

    # vamos a implementar recursive window forecasting
    
    index = oos_data.index
    end_loc = np.where(index >= split_date)[0].min()
    # esto es un int del iloc
    # preciso usar ints de iloc porque el timedelta se me va a romper con el fin de semana
    rolling_window = 252

    nstate=model.n_components
    model = hmm.GaussianHMM(n_components = nstate, **param_dict, verbose=False)

    model_list=[]
    counter=0

    for i in range(1, dates_to_forecast):
        print(i)
        date_of_first_forecast = oos_data.index[end_loc + i -1]
        
        fitstart = end_loc - rolling_window + i
        fitend = end_loc + i

        # fit model with last year
        fit_data=oos_data.iloc[fitstart:fitend][columns]
        res=model.fit(fit_data)
        model_list.append(res)
        # TODO: que pasa si fittea mal?
        
        # obtenemos las probabilidades por estado del ultimo dia
        # son las probabilidades que maximizan la log/likelihood de toda la secuencia
        index=len(model_list)
        while index>0:
            try:
                add_count=False
                last_day_state_probs = res.predict_proba(fit_data)[-1]
                probabilities.loc[date_of_first_forecast] = last_day_state_probs
                index=0

            except ValueError:
                # this happens when startprob_ must sum to 1 (got nan)
                add_count=True
                index=index-1
                res=model_list[index]
                
        if add_count:
            counter=counter+1
        # model.means_ es es la media condicional a cada estado
            # cada columna representa cada columna del dataset
            # cada fila es un estado
        # el producto punto entre este y las probabilidades del ultimo día me da la media esperada por cada columna
        expected_means = np.dot(last_day_state_probs, model.means_)
        forecasts.loc[date_of_first_forecast]=expected_means

        # si el modelo falla en el predict_proba, se utiliza el de t-1
        
    print("failed models: ", counter)
    return probabilities, forecasts

GaussianHMM(n_components=15, n_iter=500, random_state=42)

In [37]:
stock="YPFD.BA"
columns=[f"{stock}_log_rets", 
         f"{stock}_gk_vol"]

probabilities, forecasts= generate_HMM_samples_residuals(model,
                                                           insample_data=df[columns], 
                                                           oos_data=df_test[columns])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72


c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\hmmlearn\hmm.py:340: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])


73


c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\hmmlearn\hmm.py:340: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])


74


c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\hmmlearn\hmm.py:340: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])


75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
failed models:  3


In [38]:
probabilities

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
2023-06-02,0.0,0.0,0.0,0.333596,0.0,0.0,0.0,0.0,0.0,0.070566,0.303652,0.0,0.0,0.292186,0.0
2023-06-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-06-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-06-07,0.0,0.0,0.523549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.476451,0.0,0.0,0.0
2023-06-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-24,0.0,0.98376,0.0,0.0,0.0,0.0,0.0,0.000044,0.0,0.0,0.0,0.015655,0.0,0.0,0.000542
2023-11-27,0.000003,0.0,0.000288,0.0,0.0,0.0,0.0,0.999709,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-28,0.0,0.988009,0.000001,0.0,0.0,0.0,0.0,0.0,0.0,0.009505,0.001093,0.000003,0.0,0.001388,0.0
2023-11-29,0.000041,0.0,0.000021,0.0,0.0,0.0,0.0,0.999929,0.0,0.0,0.000002,0.0,0.0,0.0,0.000006


In [39]:
forecasts

,YPFD.BA_log_rets,YPFD.BA_gk_vol
2023-06-02,0.008274,0.001732
2023-06-05,0.006866,0.00176
2023-06-06,0.007185,0.001763
2023-06-07,0.00718,0.001758
2023-06-08,0.007156,0.001752
...,...,...
2023-11-24,0.006455,0.002055
2023-11-27,0.00571,0.002111
2023-11-28,0.004085,0.002192
2023-11-29,0.004857,0.002108


In [ ]:
aic_best_residuals={stock:None for stock in tickerlist}
bic_best_residuals={stock:None for stock in tickerlist}

In [ ]:
for stock in tickerlist:
    columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
    insample_data = df[columns]
    oos_data=df_test[columns]

    samples, aic_best_residuals[stock] = generate_HMM_samples_residuals(
        aic_best_model[stock], 
        insample_data=insample_data, 
        oos_data=oos_data)

    samples, bic_best_residuals[stock] = generate_HMM_samples_residuals(
        bic_best_model[stock], 
        insample_data=insample_data, 
        oos_data=oos_data)

# Guardado de datos

In [ ]:
with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_aic_bestmodels.pickle"""), "wb") as output_file:
    pickle.dump(aic_best_model, output_file)

with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_bic_bestmodels.pickle"""), "wb") as output_file:
    pickle.dump(bic_best_model, output_file)

In [ ]:
with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_aic_residuals.pickle"""), "wb") as output_file:
    pickle.dump(aic_best_residuals, output_file)

with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_bic_residuals.pickle"""), "wb") as output_file:
    pickle.dump(bic_best_residuals, output_file)

# Graficando

In [ ]:
def plot_close_rets_vol(model, data, key, IC):
    prediction= model.predict(data)
    states=set(prediction)

    fig=plt.figure(figsize = (20, 20))
    plt.tight_layout()
    plt.title(f"{key} Log returns and intraday Vol\n{model.n_components} states / best by {IC}")

    for subplot, var in zip(range(1,3), data.columns):    
        plt.subplot(2,1,subplot)
        for i in set(prediction):
            state = (prediction == i)
            x = data.index[state]
            y = data[var].iloc[state]
            plt.plot(x, y, '.')
        plt.legend(states, fontsize=16)
        
        plt.grid(True)
        plt.xlabel("datetime", fontsize=16)
        plt.ylabel(var, fontsize=16)
            
    plt.savefig(os.path.join(resultsroute, "graphs", 
                             f"HMM", 
                             f"{key}_model_{IC}.png"))

In [ ]:
for dictionary, IC in zip([aic_best_model, bic_best_model], ["AIC", "BIC"]):
    for key, model in dictionary.items():
        columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
        insample_data = df[columns]
        oos_data=df_test[columns]
        train_end=insample_data.index.max()
        data=pd.concat([insample_data, oos_data])

        plot_close_rets_vol(model, data, key, IC)

## HMM Selection

Selecting the Number of States in Hidden Markov Models: Pragmatic Solutions Illustrated Using Animal Movement
https://sci-hub.st/10.1007/s13253-017-0283-8